In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj03.ipynb")

<table style="width: 100%;">
    <tr style="background-color: transparent;"><td>
        <img src="https://data-88e.github.io/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Spring 2020<br>
            Dr. Eric Van Dusen<br>
            Notebook by Chris Pyles</p></td></tr>
</table>

# Project 3: Econometrics and Data Science

This project focuses on the application of the data science techniques from lecture. You will practice single variable ordinary least squares regression in the Data 8 style, go through a guided introduction to multivariate OLS using the package `statsmodels`, and finally create your own multivariate OLS model.

After this project, you should be able to

1. Write and apply the necesssary functions to perform single variable OLS
2. Use the `statsmodels` package to create multivariate OLS models
3. Understand how to quantitatively evaluate models using the root-mean-squared error
4. Look for and use relationships between variables to select features for regression

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import warnings

from ipywidgets import interact, Dropdown, IntSlider

warnings.simplefilter(action='ignore')
%matplotlib inline
plt.style.use('seaborn-muted')
plt.rcParams["figure.figsize"] = [10,7]

In this project, we will be working with data on credit card defaults and billing. The data covers April to September 2005, with one row for each cardholder. It has the following columns:

| Column | Description |
|-----|-----|
| `credit` | Total amount of credit |
| `sex` | Cardholder sex |
| `education` | Cardholder education level |
| `martial_status` | Cardholder marital status |
| `age` | Cardholder age |
| `bill_{month}05` | Bill amount for specific month |
| `paid_{month}05` | Amount paid in specified month |
| `default` | Whether the cardholder defaulted |

In the cell below, we load the dataset.

In [ ]:
defaults = pd.read_csv("defaults.csv")
defaults

**Question 0.1:** Which of the columns in `defaults` would we need dummies for in order to use in an OLS model? Assign `q0_1` to an list of these column _labels_.

In [ ]:
q0_1 = ...
q0_1

In [ ]:
grader.check("q0_1")

In order to use the columns you chose, we will need to create dummies for them. In lecture, we showed a function (defined in the imports cell) that will get dummies for a variable for you.

**Question 0.2:** Use `pd.get_dummies` to get dummies for the variables you listed in `q0_1`.

In [ ]:
defaults = ...

In [ ]:
grader.check("q0_2")

## Part 1: Single Variable OLS

We'll start by doing some single variable linear regression, ala Data 8. To begin, recall that we can model $y$ based on $x$ using the form

$$\Large
\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x
$$

We can define the **correlation coefficient** of two values to be the mean of the product of their values in standard units.

**Question 1.1:** Complete the `corr` function below to compute the correlation coefficient of two arrays `x` and `y` based on the formula

$$\Large
r = \text{mean} \left ( x_\text{SU} \cdot y_\text{SU} \right )
$$

_Hint:_ You may find the `su` function, which converts an array to standard units, helpful.

In [ ]:
def su(arr):
    """Converts array arr to standard units"""
    return (arr - np.mean(arr)) / np.std(arr)

def corr(x, y):
    """Calculates the correlation coefficient of two arrays"""
    ...

In [ ]:
grader.check("q1_1")

From this $r$ value that we have calculated above, we can compute the slope $\beta_1$ and intercept $\beta_0$ of the best-fit line using the formulas below.

$$\Large
\beta_1 = r \frac{\hat{\sigma}_y}{\hat{\sigma}_x}
\qquad \text{ and } \qquad
\beta_0 = \hat{\mu}_y - \beta_1 \cdot \hat{\mu}_x
$$

**Question 1.2:** Using your `corr` function, fill in the `slope` and `intercept` functions below which compute the values of $\beta_1$ and $\beta_0$ for the line of best fit that predicts `y` based on `x`. Your function should use vectorized arithmetic (i.e. no `for` loops).

_Hint:_ You may find your `slope` function useful in `intercept`.

In [ ]:
def slope(x, y):
    """Computes the slope of the best-fit line of y based on x"""
    ...

def intercept(x, y):
    """Computes the intercept of the best-fit line of y based on x"""
    ...

In [ ]:
grader.check("q1_2")

Now let's look at how we can predict the `bill_sep05` column based on some other column of our data. We'll start by looking at the `credit` as the explanatory variable. To use our functions above, we must extract the values of each column as arrays, which we define below as `x` and `y`. We then compute the fitted values `y_hat` using the slope-intercept formula and plot the results.

**Question 1.3:** Using the functions you defined in Question 1.2, regress `bill_sep05` on `credit`. Assign your predictions to `y_hat`.

In [ ]:
x = defaults["credit"]
y = defaults["bill_sep05"]

beta_1 = ...
beta_0 = ...
y_hat = ...

In [ ]:
grader.check("q1_3")

Now that we have some predictions, let's plot the original data and the regression line.

In [ ]:
plt.scatter(x, y, color="tab:blue", alpha=0.3)
plt.plot(x, y_hat, color="tab:red")
plt.title("Predict September bill with credit");

<!-- BEGIN QUESTION -->

**Question 1.4:** Does the line we found fit the data well? Explain.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

Let's estimate how confident we are in the significance of our $\hat{\beta}_1$ coefficient.

**Question 1.5:** Fill in the code below to bootstrap our $\hat{\beta}_1$ and find the 95% confidence interval. Store the lower and upper bounds as `ci95_lower` and `ci95_upper`, respectively. (The cell may take a couple minutes to run.)

_Hint:_ Since we're only interested in $\hat{\beta}_1$, we don't need to find the intercept or fit our $x$ values.

In [ ]:
betas = []

for i in np.arange(200):
    sample = defaults.sample(5000)    # defaults is a huge table, so we'll only sample 5000 rows
    sample_x = ...
    sample_y = ...
    ...

ci95_lower = ...
ci95_upper = ...

print("95% CI: ({}, {})".format(ci95_lower, ci95_upper))

In [ ]:
grader.check("q1_5")

<!-- BEGIN QUESTION -->

**Question 1.6:** Using your 95% confidence interval, is it likely that the credit has no effect on the September 2005 bill? Justify your answer.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

Obviously, we can see that our best-fit line does not predict perfectly. There are plenty of points in the scatterplot that do not fall on the line. But how do we quantify the error of our model? There are many so-called *loss functions*, but in this notebook we will use the **root-mean-squared error**, which is defined as

$$\Large
\text{RMSE} = \sqrt{ \frac{1}{n} \sum_{i=1}^n \left ( y_i - \hat{y}_i \right ) ^2 }
$$

where $n$ is the number of observations. The effect of this is to take the mean of the distance of each value of $\hat{y}$ from its corresponding value in $y$; squaring these values keeps them positive, and then we take the square root to correct the units of the error.

**Question 1.7:** Complete the function `rmse` below which computes the root-mean-squared error of the prediction `y_hat` on `y`. Again, no `for` loops.

In [ ]:
def rmse(y, y_hat):
    """Computes the RMSE of prediction y_hat based on y"""
    ...

In [ ]:
grader.check("q1_7")

**Question 1.8:** Use your `rmse` function to compute the RMSE of our prediction `y_hat` based on `y` from above.

In [ ]:
single_var_error = ...
single_var_error

In [ ]:
grader.check("q1_8")

Now that we know how to predict based on and quantify the error of a model, let's write a function that will encapsulate this pipeline for us.

<!-- BEGIN QUESTION -->

**Question 1.9:** Fill in the function `pred_and_plot` below which models `bill_sep05` based on a column `col`, plots the scatterplot and line of best fit, and computes the RMSE of the model. Then choose a column you think might be related to `bill_sep05` and use your `pred_and_plot` function to determine its prediction RMSE and plot the regression line.

_Hint:_ Your code from Question 1.3 may be helpful here...

In [ ]:
def pred_and_plot(col):
    """Performs single variable OLS to predict bill_sep05 based on col"""
    x = ...
    y = ...

    beta_1 = ...
    beta_0 = ...

    y_hat = ...
    
    model_rmse = ...
    
    
    ### DO NOT EDIT THE REST OF THIS FUNCTION ###
    print("RMSE: {:.5f}".format(rmse(y, y_hat)))

    plt.scatter(x, y, color="tab:blue", alpha=0.3)
    plt.plot(x, y_hat, color="tab:red")
    plt.title("Predict September bill with {}".format(col))

### Provide your column name below ###
pred_and_plot(...)
...

<!-- END QUESTION -->

In looking through different features, you should have noticed that most of them don't follow a linear relationship very well. In practice, you often need _multiple_ features (explanatory variables) to predict an outcome variable, and it is for this reason that we often use **multiple linear regression** to predict variables.

Finally, before moving on to the multivariable case, let's think about using whether or not an individual defaults as a predictor of their September 2005 bill.

**Question 1.10:** Assign `default_beta_1` and `default_beta_0` to the slope and intercept of your regression of `bill_sep05` on the `default` column of the table `defaults`.

_Hint:_ Our outcome variable hasn't changed, so we can reuse the array `y` defined earlier.

In [ ]:
default_x = ...

default_beta_1 = ...
default_beta_0 = ...

print("y_hat = {} * x + {}".format(default_beta_1, default_beta_0))

In [ ]:
grader.check("q1_10")

<!-- BEGIN QUESTION -->

**Question 1.11:** Interpret the value of `default_beta_1`. Basically, what do we expected to happen when `default` changes from 0 to 1? Explain.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Part 2: Guided Multivariable OLS

When we predict a variable $y$ based on some set of $p$ explanatory variables $x$, we create a model of the world with set of weights $\left \{ \beta_i \right \}$ such that we have

$$\Large
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_p x_p + \varepsilon
$$

Because of the error term $\varepsilon$, we will instead create predictions $\hat{y}$, such that 

$$\Large
\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x_1 + \hat{\beta}_2 x_2 + \cdots + \hat{\beta}_p x_p
$$

Let's model the September bill based on the other bills in the data set (April to August). Recall from lecture that we can model an outcome variable `Y` based on columns from our data `defaults` by extracting the values of the table into an array. In the cell below, we create the arrays `X` and `Y`.

In [ ]:
X = defaults[["bill_aug05", "bill_jul05", "bill_jun05", "bill_may05", "bill_apr05"]]
Y = defaults["bill_sep05"]

Recall that we can fit a multivariate OLS model using `statsmodels` by calling the function `sm.OLS` on the outcome and explanatory variables. In the cell below, we create a model based on _all_ the columns in the table (except, of course, the outcome variable).

In [ ]:
# create an OLS object with the data
model = sm.OLS(Y, sm.add_constant(X))
result = model.fit()
result.summary()

**Question 2.1:** What is the standard error of the coefficient of `bill_jun05`?

<ol style="list-style-type: lower-alpha;">
    <li>0.005</li>
    <li>0.010</li>
    <li>0.039</li>
    <li>0.007</li>
</ol>

Assign your answer to `q2_1` below.

In [ ]:
q2_1 = ...

In [ ]:
grader.check("q2_1")

<!-- BEGIN QUESTION -->

**Question 2.2:** Which bills are likely good predictors of `bill_sep05`? Justify your response.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

Now let's look and see what values our model predicts for our outcome variable. Recall that we can extract the fitted values from the result using `result.fittedvalues`. 

**Question 2.3:** Assign `Y_hat` to the fitted values of `result`. Then assign `multi_rmse` to the RMSE of this prediction based on `Y`.

In [ ]:
Y_hat = ...

multi_rmse = ...
multi_rmse

In [ ]:
grader.check("q2_3")

We see from this RMSE that the prediction is (much) better than the single variable case, but it's still not too good. Let's try and select better features to see if we can lower our RMSE.

**Question 2.4:** Add one more column label to the array `new_features` below. Then fill in the code below to create a new OLS model based on the columns in `new_features`, storing the fitted values in `new_Y_hat`. **Don't forget to apply `sm.add_constant` to `new_X` in your `sm.OLS` call!**

_Hint:_ Our outcome variable `Y` hasn't changed, so we can reuse the same array as earlier.

In [ ]:
new_features = ["bill_aug05", "bill_jul05", "paid_aug05", "paid_jul05", "sex_male", ...]

new_X = ...

new_model = ...
new_result = ...
new_Y_hat = ...
new_Y_hat

In [ ]:
grader.check("q2_4")

Now that we have some predictions, let's look at the accuracy of our model.

**Question 2.5:** Calculate the RMSE of `new_Y_hat` based on `Y` and store this value as `new_rmse`.

In [ ]:
new_rmse = ...
new_rmse

In [ ]:
grader.check("q2_5")

<!-- BEGIN QUESTION -->

**Question 2.6:** Did the RMSE go up or down in Question 2.7 compared to Question 2.4? Why do you think so?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Part 3: Unguided Multivariable OLS

In this section of the assignment, you will use `statsmodels` and OLS to create your own model to predict the September 2005 bill. Your model will be scored out of **5 points**, and a portion of your score will be determined based on your RMSE. The scores you will receive are given in the table below.

| RMSE | Score (out of 5) |
|-----|-----|
| $\le$ 20,000 | 6 |
| $\le$ 30,000 | 5 |
| $\le$ 50,000 | 4 |
| $\le \infty$ | 3 |

Note that it is possible to receive a 6 out of 5 for an especially good model, and that as long as you *create a model*, you are guaranteed a 3 out of 5. **To submit your model, you must assign `my_labels` to an array of the columns you want your model to use. You may not use more than 10 columns and, of course, you can't use the column `bill_sep05` in your features.** Your model RMSE will be calculated using the following code:

```python
X, Y = defaults[my_labels], defaults["bill_sep05"]
model = sm.OLS(Y, sm.add_constant(X))
result = model.fit()
Y_hat = result.fittedvalues
rmse(Y, Y_hat)
```

To select your features, use the widget below to look for correlations between variables and the September 2005 bill. It requires your `pred_and_plot` function to work, so you will need to finish that function before using the widget.

In [ ]:
interact(pred_and_plot, col=Dropdown(options=defaults.columns));

Add and remove cells below as needed, but *make sure you define `my_labels`*. We have provided code for you to create your `X` array; just fill in the `...` in `my_labels` with your columns and use the space at the bottom to work on your model. Good luck!

In [ ]:
my_labels = [...]

my_X = defaults[my_labels]

my_model = ...
my_result = ...
my_Y_hat = ...
rmse(...)

In [ ]:
grader.check("q3")

## Part 4: Reflection

In this section of the assignment, you will answer some conceptual questions about the choices you made in creating your model in Part 3. **This section heavily influences your grade, as we are looking to ensure that you are using econometric intuition while modeling. Please answer thoughtfully and, as always, *show us the numbers*.**

<!-- BEGIN QUESTION -->

**Question 4.1:** Explain one choice you made in selecting features while modeling in Part 3 and why you made it. (Your explanation should take at least a few sentences, and should justify your choice mathematically (i.e. with numerical evidence).)

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 4.2:** Use your `pred_and_plot` function in the cell below to generate a visualization that helped you choose a feature in Part 3.

In [ ]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 4.3:** Choose a column you regressed on. Report its coefficient, $t$ statistic, and 95% CI. Interpret the coefficient's value. Is the variable likely significant? Explain.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

---

### References

* Data from https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)